In [316]:
# Import the required packages
import requests
import pprint
import duckdb
import pandas as pd

In [319]:
# Open a connection to the persistent database
con = duckdb.connect('test.duckdb')

con.sql("SELECT * FROM duckdb_tables()") # Note: Don't use a semi colon in DuckDB for Python

┌───────────────┬──────────────┬─────────────┬────────────┬────────────┬───────────┬─────────┬───────────────────────┬──────────┬───────────┬─────────────────┬────────────────┬──────────────┬─────────────┬────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [309]:
# Store the base API information, for this, I'm using NASA's Near Earth Object (NEO) API
api_key_path = "/Users/chriskornaros/Documents/local-scripts/.api_keys/nasa/key.txt"
with open(api_key_path, 'r') as file:
    api_key = file.read().strip()
start_date, end_date = "2024-12-01", "2024-12-04"

# Then Construct the URL with f-string formatting
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [310]:
# Standalone cell for the API response, ensures I don't call too many times
response = requests.get(url)

# Check if the response was successful, if so, print the data in a human readable format
if response.status_code == 200:
    data = response.json()
    pprint.pprint(data)
else:
    print(f"Error: {response.status_code}")

{'element_count': 6,
 'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-12-04&end_date=2024-12-07&detailed=false&api_key=a9EgkXapvgaGdluVVB9VlbQaDWF68gB4czBZd8NK',
           'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-11-28&end_date=2024-12-01&detailed=false&api_key=a9EgkXapvgaGdluVVB9VlbQaDWF68gB4czBZd8NK',
           'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-12-01&end_date=2024-12-04&detailed=false&api_key=a9EgkXapvgaGdluVVB9VlbQaDWF68gB4czBZd8NK'},
 'near_earth_objects': {'2024-12-02': [{'absolute_magnitude_h': 17.56,
                                        'close_approach_data': [{'close_approach_date': '2024-12-02',
                                                                 'close_approach_date_full': '2024-Dec-02 '
                                                                                             '12:36',
                                                                 'epoch_date_close_approach': 17

In [313]:
# Flatten the output and then read it with DuckDB to get the column data types
flat_data = pd.json_normalize(data['near_earth_objects']['2024-12-02'])

raw = con.sql("""CREATE TABLE asteroids AS
                SELECT * EXCLUDE(nasa_jpl_url, close_approach_data, 'links.self')
                , unnest(close_approach_data, recursive := true)
                FROM flat_data
                """)

In [321]:
con.close()